In [1]:
import sys
import pandas as pd 
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import IPython
import sklearn 
import xgboost as xgb

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

X = pd.read_csv("../data/preprocessed/trainig_data.csv")

# Копия, чтобы не портить исходные данные
X_fe = X.copy()

# 2. Индекс "жарко-влажно"
X_fe["temp_humidity_index"] = X_fe["temperature_C"] * X_fe["humidity_%"]

# 3. Полиномиальные признаки
X_fe["soil_pH_squared"] = X_fe["soil_pH"] ** 2
X_fe["temperature_squared"] = X_fe["temperature_C"] ** 2

# 4. Комбинации категориальных
X_fe["region_croptype"] = X_fe["region_le"] * X_fe["croptype_le"]
X_fe["fert_irrigation"] = X_fe["fertilizer_type_le"] * X_fe["irrigation_type_le"]

# 5. Географические преобразования
X_fe["lat_lon_ratio"] = X_fe["latitude"] / (X_fe["longitude"] + 1e-6)
X_fe["lat_lon_sum"]   = X_fe["latitude"] + X_fe["longitude"]

# 6. Кластеры по координатам (например, 3 региона)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
X_fe["geo_cluster"] = kmeans.fit_predict(X_fe[["latitude", "longitude"]])

# 7. Логарифмы (для скошенных признаков)
X_fe["log_rainfall"] = np.log1p(X_fe["rainfall_mm"])
X_fe["log_pesticide"] = np.log1p(X_fe["pesticide_usage_ml"])

print("Новые размеры:", X.shape, "→", X_fe.shape)

# --- Блок feature engineering для болезней ---

# Флаг: есть болезнь или нет
X_fe["has_disease"] = (X_fe["cropdisease_le"] > 0).astype(int)

# Взаимодействия болезни с погодой
X_fe["disease_humidity"] = X_fe["cropdisease_le"] * X_fe["humidity_%"]
X_fe["disease_temperature"] = X_fe["cropdisease_le"] * X_fe["temperature_C"]

# Болезнь × удобрение
X_fe["disease_fertilizer"] = X_fe["cropdisease_le"] * X_fe["fertilizer_type_le"]


X_fe.head()




c:\Users\esimb\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Новые размеры: (500, 17) → (500, 27)


,Unnamed: 0,soil_moisture_%,soil_pH,temperature_C,rainfall_mm,humidity_%,sunlight_hours,pesticide_usage_ml,latitude,longitude,...,fert_irrigation,lat_lon_ratio,lat_lon_sum,geo_cluster,log_rainfall,log_pesticide,has_disease,disease_humidity,disease_temperature,disease_fertilizer
0,0,35.95,5.99,17.79,75.62,77.03,7.27,6.34,14.970941,82.997689,...,4,0.180378,97.968630,0,4.338858,1.993339,0,0.00,0.00,0
1,1,19.74,7.24,30.18,89.91,61.13,5.67,9.60,16.613022,70.869009,...,0,0.234419,87.482031,0,4.509870,2.360854,1,122.26,60.36,0
2,2,29.32,7.16,27.37,265.43,68.87,8.23,15.26,19.503156,79.068206,...,0,0.246662,98.571362,0,5.585112,2.788708,0,0.00,0.00,0
3,3,17.33,6.03,33.73,212.01,70.46,5.03,25.80,31.071298,85.519998,...,6,0.363322,116.591296,2,5.361339,3.288402,1,140.92,67.46,4
4,4,19.37,5.92,33.86,269.09,55.73,7.93,25.65,16.568540,81.691720,...,2,0.202818,98.260260,0,5.598755,3.282789,1,167.19,101.58,3


In [8]:
X_fe.to_csv("../data/preprocessed/training.data.csv")